In [3]:
!pip install pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.getOrCreate()

In [8]:
df1 = spark.read.csv('./Dataset/taxi_zone_lookup.csv', header=True)

In [20]:
df1.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [10]:
df2 = spark.read.parquet('./Dataset/BigTable.parquet', header=True)

In [11]:
df2.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [12]:
df2.columns

['hvfhs_license_num',
 'dispatching_base_num',
 'originating_base_num',
 'request_datetime',
 'on_scene_datetime',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_miles',
 'trip_time',
 'base_passenger_fare',
 'tolls',
 'bcf',
 'sales_tax',
 'congestion_surcharge',
 'airport_fee',
 'tips',
 'driver_pay',
 'shared_request_flag',
 'shared_match_flag',
 'access_a_ride_flag',
 'wav_request_flag',
 'wav_match_flag']

In [15]:
df1.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [18]:
df2.select('PULocationID', 'DOLocationID').show()

+------------+------------+
|PULocationID|DOLocationID|
+------------+------------+
|         138|         141|
|          61|          80|
|          36|         260|
|          80|          42|
|         152|          41|
|         138|          92|
|         164|          17|
|          37|         225|
|         225|          79|
|          17|          49|
|          17|         148|
|         255|          65|
|          33|         255|
|         255|         112|
|         230|          68|
|          68|         158|
|         249|          17|
|         167|         167|
|          69|         147|
|         147|         212|
+------------+------------+
only showing top 20 rows


In [40]:
df2=df2.select('pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_miles',
 'trip_time',
 'driver_pay')

In [24]:
df2.select('pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_miles',
 'trip_time',
 'driver_pay').show(5)

+-------------------+-------------------+------------+------------+----------+---------+----------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|driver_pay|
+-------------------+-------------------+------------+------------+----------+---------+----------+
|2024-07-01 00:19:43|2024-07-01 00:40:35|         138|         141|      8.84|     1252|     24.19|
|2024-07-01 00:21:00|2024-07-01 00:41:24|          61|          80|      3.96|     1224|     19.37|
|2024-07-01 00:45:10|2024-07-01 01:01:05|          36|         260|      5.61|      955|     12.08|
|2024-07-01 00:49:01|2024-07-01 01:15:02|          80|          42|     11.24|     1561|     25.64|
|2024-07-01 00:41:36|2024-07-01 00:49:48|         152|          41|       1.6|      492|      7.11|
+-------------------+-------------------+------------+------------+----------+---------+----------+
only showing top 5 rows


In [25]:
df1.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows


In [28]:
print('Main DataFrame Count : ', df2.count())
print('Location DataFrame Count : ', df1.count())

Main DataFrame Count :  19182934
Location DataFrame Count :  265


Requirement : We need a final table with Zone names instead of location Id for data analysis

As data volume in DF2 is very high and data volume in DF1 is very less. We will use a query optimization technique, "Broadcast Join"

In [30]:
df2.rdd.getNumPartitions()

8

In [36]:
from pyspark.sql.functions import broadcast

In [42]:
df2_alias = df2.alias("df2")
pickup_df1 = df1.alias("pickup")
dropoff_df1 = df1.alias("dropoff")

In [58]:
df = df2_alias.join(broadcast(pickup_df1), on=df2_alias.PULocationID == pickup_df1.LocationID, how="inner") \
    .join(broadcast(dropoff_df1), on=df2_alias.DOLocationID == dropoff_df1.LocationID, how="inner") \
    .selectExpr("df2.*","pickup.Zone as PickUp_Zone","dropoff.Zone as DropOff_Zone")

In [57]:
df.show(5)

+-------------------+-------------------+------------+------------+----------+---------+----------+-------------------+--------------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|driver_pay|        PickUp_Zone|        DropOff_Zone|
+-------------------+-------------------+------------+------------+----------+---------+----------+-------------------+--------------------+
|2024-07-01 00:19:43|2024-07-01 00:40:35|         138|         141|      8.84|     1252|     24.19|  LaGuardia Airport|     Lenox Hill West|
|2024-07-01 00:21:00|2024-07-01 00:41:24|          61|          80|      3.96|     1224|     19.37|Crown Heights North|   East Williamsburg|
|2024-07-01 00:45:10|2024-07-01 01:01:05|          36|         260|      5.61|      955|     12.08|     Bushwick North|            Woodside|
|2024-07-01 00:49:01|2024-07-01 01:15:02|          80|          42|     11.24|     1561|     25.64|  East Williamsburg|Central Harlem North|
|2024-07-01 0